In [188]:
import requests
import lxml
from bs4 import BeautifulSoup
# 에러 나면 pip install openpyxl
from openpyxl import Workbook, load_workbook
import json
import random # for numguess

## 정리한 요소 저장하기

In [4]:
# n사 메인페이지 접속하기
response = requests.get("https://www.naver.com/")
response.status_code

200

In [7]:
# 응답으로 받은 텍스트값 page_text에 할당
page_text = response.text
page_text[:200]

'<!doctype html>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equi'

In [11]:
# 전체 소스에서 검색어에 해당하는 부분 찾기
soup = BeautifulSoup(page_text, "lxml")
ul_text = soup.find("ul", attrs={"class":"ah_l"})
ul_text

<ul class="ah_l">
<li class="ah_item">
<a class="ah_a" data-clk="lve.keyword" href="#">
<span class="ah_r">1</span>
<span class="ah_k">일본 태풍</span>
</a>
</li>
<li class="ah_item">
<a class="ah_a" data-clk="lve.keyword" href="#">
<span class="ah_r">2</span>
<span class="ah_k">박정현 샹들리에</span>
</a>
</li>
<li class="ah_item">
<a class="ah_a" data-clk="lve.keyword" href="#">
<span class="ah_r">3</span>
<span class="ah_k">프레시지 삼시세끼</span>
</a>
</li>
<li class="ah_item">
<a class="ah_a" data-clk="lve.keyword" href="#">
<span class="ah_r">4</span>
<span class="ah_k">링링</span>
</a>
</li>
<li class="ah_item">
<a class="ah_a" data-clk="lve.keyword" href="#">
<span class="ah_r">5</span>
<span class="ah_k">겟아웃</span>
</a>
</li>
<li class="ah_item">
<a class="ah_a" data-clk="lve.keyword" href="#">
<span class="ah_r">6</span>
<span class="ah_k">박정현</span>
</a>
</li>
<li class="ah_item">
<a class="ah_a" data-clk="lve.keyword" href="#">
<span class="ah_r">7</span>
<span class="ah_k">컨시크올데이커버쿠션</span>
<

In [15]:
# 전체 단위를 순위를 기준으로 잘라 li_list에 할당
li_list = ul_text.find_all("li")
li_list[0]

<li class="ah_item">
<a class="ah_a" data-clk="lve.keyword" href="#">
<span class="ah_r">1</span>
<span class="ah_k">일본 태풍</span>
</a>
</li>

In [17]:
# 검색어 하나에 대해 순위와 키워드 찾아내 nv_querys에 저장
nv_querys = []
for li in li_list:
    rank = li.find("span", attrs={"class":"ah_r"}).text
    keyword = li.find("span", attrs={"class":"ah_k"}).text
    nv_querys.append((rank, keyword))

In [18]:
nv_querys

[('1', '일본 태풍'),
 ('2', '박정현 샹들리에'),
 ('3', '프레시지 삼시세끼'),
 ('4', '링링'),
 ('5', '겟아웃'),
 ('6', '박정현'),
 ('7', '컨시크올데이커버쿠션'),
 ('8', '김연아'),
 ('9', '이태원 지구촌 축제'),
 ('10', '이재황'),
 ('11', '샹들리에'),
 ('12', '그랜절'),
 ('13', '가수 강남 집안'),
 ('14', '영화 겟아웃'),
 ('15', '김필'),
 ('16', '유다솜'),
 ('17', '비긴어게인3'),
 ('18', '경주백설갈비찜'),
 ('19', '헨리'),
 ('20', '왕밤빵')]

In [22]:
with open('nv_querys.csv', 'a') as f:
    for kw in nv_querys:
        f.write(kw[0]+","+kw[1]+"\n")

In [23]:
with open('nv_querys.csv', 'r') as f:
    data = f.readlines()
    for text in data:
        print(text)

1,일본 태풍2,박정현 샹들리에3,프레시지 삼시세끼4,링링5,겟아웃6,박정현7,컨시크올데이커버쿠션8,김연아9,이태원 지구촌 축제10,이재황11,샹들리에12,그랜절13,가수 강남 집안14,영화 겟아웃15,김필16,유다솜17,비긴어게인318,경주백설갈비찜19,헨리20,왕밤빵1,일본 태풍

2,박정현 샹들리에

3,프레시지 삼시세끼

4,링링

5,겟아웃

6,박정현

7,컨시크올데이커버쿠션

8,김연아

9,이태원 지구촌 축제

10,이재황

11,샹들리에

12,그랜절

13,가수 강남 집안

14,영화 겟아웃

15,김필

16,유다솜

17,비긴어게인3

18,경주백설갈비찜

19,헨리

20,왕밤빵



In [32]:
workbook_name = 'nv_querys.xlsx'

try:
    workbook = load_workbook(workbook_name)
except FileNotFoundError as e:
    workbook = Workbook()

worksheet = workbook.active
for rank, keyword in nv_querys:
    worksheet.append((int(rank), keyword))

workbook.save(workbook_name)

In [40]:
file_name = "nv_querys.json"

nv_object = dict(nv_querys)

try:
    with open(file_name, 'r+') as f:
        data = json.load(f)
        data["data"].append(nv_object)
        f.seek(0)
        json.dump(data, f)
        f.truncate()
except FileNotFoundError as e: # -> e가 빠졌..
    with open(file_name, 'w') as f:
        json.dump({"data":[nv_object]}, f) # -> f가 빠졌..

In [36]:
nv_object

{'1': '일본 태풍',
 '2': '박정현 샹들리에',
 '3': '프레시지 삼시세끼',
 '4': '링링',
 '5': '겟아웃',
 '6': '박정현',
 '7': '컨시크올데이커버쿠션',
 '8': '김연아',
 '9': '이태원 지구촌 축제',
 '10': '이재황',
 '11': '샹들리에',
 '12': '그랜절',
 '13': '가수 강남 집안',
 '14': '영화 겟아웃',
 '15': '김필',
 '16': '유다솜',
 '17': '비긴어게인3',
 '18': '경주백설갈비찜',
 '19': '헨리',
 '20': '왕밤빵'}

In [39]:
with open('nv_querys.json', 'r') as f:
    data = json.load(f)
    print(data)

{'data': [{'1': '일본 태풍', '2': '박정현 샹들리에', '3': '프레시지 삼시세끼', '4': '링링', '5': '겟아웃', '6': '박정현', '7': '컨시크올데이커버쿠션', '8': '김연아', '9': '이태원 지구촌 축제', '10': '이재황', '11': '샹들리에', '12': '그랜절', '13': '가수 강남 집안', '14': '영화 겟아웃', '15': '김필', '16': '유다솜', '17': '비긴어게인3', '18': '경주백설갈비찜', '19': '헨리', '20': '왕밤빵'}]}


In [ ]:
# CSV : a,b,c,d
# excel: table
# json: {a:1, b:2, c:3}

## Python 기본문법
### Operation, Operators

In [41]:
# + - * /
3+2

2-3

17*4

7/2

5

In [45]:
3 2 /

SyntaxError: invalid syntax (<ipython-input-45-9145b16c209a>, line 1)

In [46]:
8/2

4.0

In [47]:
7/2

3.5

In [48]:
7//2

7%2

3**2

3

In [52]:
type(3 > 2) # 비교연산자: 비교의 결과를 참과 거짓으로 표현

bool

In [53]:
3 > 2

True

In [54]:
3 < 2

False

In [55]:
10 == 10

True

In [56]:
10 == 9

False

In [57]:
10 != 9

True

### Variable

In [59]:
a = 10 # = 는 assignment operator로 오른쪽의 값을 왼쪽의 변수명에 할당합니다
b = 5

In [60]:
a + b

15

In [61]:
print(a,b)

10 5


In [64]:
type(10)

int

In [65]:
type(3.5)

float

In [66]:
"왕밤빵"

'왕밤빵'

In [67]:
type("왕밤빵") # string (character)(character)(character)(character)

str

In [68]:
왕밤빵 # 글자가 "" 또는 '' 로 둘러싸여있다면 str, 아니면 변수이름!

NameError: name '왕밤빵' is not defined

In [69]:
hello = 1
greeting = "hello"

In [70]:
hello

1

In [71]:
greeting

'hello'

#### Type casting

In [74]:
print(int("1") + 1)

2


In [75]:
int("안녕하세요")

ValueError: invalid literal for int() with base 10: '안녕하세요'

In [76]:
float(1)

1.0

In [78]:
int(3.7) + 1

4

In [79]:
# tuple과 list
animals = ["cat", "dog", "hamster", "acorn"]

In [81]:
animals[0]

'cat'

In [82]:
animals = ("cat","dog")
animals.append("hamster") # tuple과 list의 차이: 값을 더 추가할 수 없음

AttributeError: 'tuple' object has no attribute 'append'

In [84]:
# Set, Dictionary
pokeball = {'pikachu', 'pairi', 'kkobugi', 'pikachu'}
pokeball

{'kkobugi', 'pairi', 'pikachu'}

In [91]:
words = ("a", "b", "a", "a")
words = tuple(set(words))

In [92]:
words

('b', 'a')

In [93]:
{"1","2","3"} # set은 자료형 내부에 값이 나열되어 있음

{'1', '2', '3'}

In [94]:
hong = {
    "name": "gil-dong, Hong",
    "phone": "+821012341234",
    "location": "Seoul, Korea" # dict는 key, value가 존재
}

In [95]:
hong["name"]

'gil-dong, Hong'

In [96]:
hong = ["홍길동", "+82101010101", "Seoul, Korea"]
hong[0]

'홍길동'

In [103]:
print("hello","world", sep='\n', end='\t')
print("hello, python")

hello
world	hello, python


In [104]:
print? #도움말

In [105]:
input("안녕하세요. 당신의 이름은?") # 사용자 입력

안녕하세요. 당신의 이름은?홍길동


'홍길동'

In [111]:
your_name = input("안녕하세요. 당신의 이름은?")

안녕하세요. 당신의 이름은?12


In [112]:
your_name

'12'

In [113]:
type(your_name)

str

In [116]:
your_age = int(input("당신의 나이는 얼마입니까?"))

당신의 나이는 얼마입니까?12


In [117]:
your_age > 19

False

In [121]:
pi = 3.1415

r = int(input("계산하고 싶은 반지름을 입력하세요 Human >"))

gi_reum = 2 * r
dul_re = 2 * pi * r
nul_be = pi * r ** 2
volume = 4 / 3 * pi * r ** 3
print(gi_reum, dul_re, nul_be, volume, sep='\n')

계산하고 싶은 반지름을 입력하세요 Human >10
20
62.830000000000005
314.15000000000003
4188.666666666666


In [122]:
some_string = "Today, I'm not doing anyting. And python is great language."

In [125]:
some_string.count("Today")

1

In [126]:
some_string.find("not")

11

In [130]:
result = "fast".join("안녕하세요")
result

'안fast녕fast하fast세fast요'

In [131]:
result.split("fast")

['안', '녕', '하', '세', '요']

In [132]:
"오늘은 날씨가 참 좋군요.".split(" ")

['오늘은', '날씨가', '참', '좋군요.']

In [133]:
"오늘은 날씨가 참 좋군요.".replace("날씨가", "기부니가")

'오늘은 기부니가 참 좋군요.'

In [134]:
"    안녕하세요     ".strip()

'안녕하세요'

In [135]:
"    .....!!!!안녕하세요,,,,////     ".strip(" .!/,")

'안녕하세요'

In [136]:
names = ["a", "b", "c"]
companys = ["dd", "ee", "ff"]

In [141]:
# String formating

# Level1 : 한문장 한문장 손수 만들기
#"안녕하세요 dd 의 a 님. 오랜만에 연락드립니다."
#"안녕하세요 ee 의 b 님. 오랜만에 연락드립니다."
#"안녕하세요 ff 의 c 님. 오랜만에 연락드립니다."

# Level2 : format 기능으로 채워넣기를 손수 하기
#"안녕하세요 {} 의 {} 님. 오랜만에 연락드립니다.".format("dd", "a")

# Level3 : 이름과 회사 리스트를 미리 작성해두고 자동화시키기

print(list(zip(names, companys))) # zip을 하면 두개의 리스트를 합칠 수 있습니다
for name, company in zip(names, companys): # 합친 리스트의 앞 뒷값으로
    print("안녕하세요 {0} 의 {1} 님. 오랜만에 연락드립니다.".format(company, name))

[('a', 'dd'), ('b', 'ee'), ('c', 'ff')]
안녕하세요 dd 의 a 님. 오랜만에 연락드립니다.
안녕하세요 ee 의 b 님. 오랜만에 연락드립니다.
안녕하세요 ff 의 c 님. 오랜만에 연락드립니다.


In [152]:
animals = []
animals

[]

In [153]:
animals.append("cat")

In [154]:
animals

['cat']

In [155]:
animals = ["dog", "hamster"]
animals

['dog', 'hamster']

In [156]:
animals.insert(1, "cat")
animals

['dog', 'cat', 'hamster']

In [157]:
animals.index("cat")

1

In [160]:
animals[1]

'cat'

In [161]:
animals.remove("hamster")
animals

['dog', 'cat']

In [162]:
some_animal = animals.pop()
some_animal

'cat'

In [163]:
animals

['dog']

In [164]:
animals = ["dog", "cat", "hamster", "aligator", "cameleon"]

In [165]:
third_animal = animals.pop(2)
third_animal

'hamster'

In [166]:
animals

['dog', 'cat', 'aligator', 'cameleon']

In [167]:
result = animals.pop(animals.index("aligator"))
result

'aligator'

In [171]:
numbers = [1, 100, 67, -100, 3.14, 2.71828, 1.414]

In [172]:
numbers.sort()
numbers

[-100, 1, 1.414, 2.71828, 3.14, 67, 100]

In [173]:
numbers.reverse()
numbers

[100, 67, 3.14, 2.71828, 1.414, 1, -100]

In [174]:
numbers = [1, 100, 67, -100, 3.14, 2.71828, 1.414]
numbers.sort(reverse=True)
numbers

[100, 67, 3.14, 2.71828, 1.414, 1, -100]

## 조건문

In [177]:
#if condition:
#    statement

if 3 > 2:
    print("3 is bigger than 2.")

3 is bigger than 2.


In [178]:
if 3 < 2:
    print("3 is bigger than 2.")
else:
    print("something wrong..")

something wrong..


In [180]:
user_num = int(input("pick the num 1 to 100:"))

if user_num % 2 == 0:
    print("Even")
else:
    print("Odd")

pick the num 1 to 100:9
Odd


In [187]:
user_num = int(input("pick the num 1 to 100:"))

if user_num % 2 == 0:
    print("Even")
else:
    if user_num % 3 == 0:
        print("times of 3")
    else:
        print("2의 배수도, 3의 배수도 아님.")

pick the num 1 to 100:6
Even


In [ ]:
user_num = int(input("pick the num 1 to 100:"))

if user_num % 2 == 0:
    print("Even")
elif user_num % 3 == 0:
    print("3의 배수")
elif user_num % 5 == 0:
    print("5의 배수")
else:
    print("2의 배수도, 3의 배수도 아님.")

## Numguess

In [201]:
random.randint(1,100)

62

In [208]:
answer = random.randint(1,100)
print(answer)

# 사용자로부터 값을 입력받아야 함
guess = int(input("1부터 100 사이의 정수를 입력하십시오 Human."))
print(type(guess))
# 그 값으로 정답과 비교하는 조건문 하나
if guess == answer:
    print("정답입니다.")
else:
    print("틀렸으니 1BTC를 전송하시오.")

17
1부터 100 사이의 정수를 입력하십시오 Human.17
<class 'int'>
정답입니다.


In [ ]:
# 숙제1: numguess에 추가하고싶은 기능 생각해보기(2개정도)

# 숙제2: 사용자가 1~100 사이의 정수를 입력할때 
# 그 값이 3의 배수 일때는 "fizz"
# 5의 배수 일때는 "buzz"
# 15의 배수일때는 "fizzbuzz"
# 나머지 경우는 그냥 그 수를 출력해주세요